In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import requests
import s3fs
import pandas as pd

from datetime import timedelta, datetime
import pendulum

from prefect import Client
from prefect.schedules import IntervalSchedule
from prefect.environments.storage import Docker


from extractMOCData import CONFIG as cfg
from extractMOCData.moc_data import TsxMocData
#from extractMOCData import read_moc 

In [4]:
tsxMocData = TsxMocData()

moc_df = tsxMocData.scrape_moc_data()

In [5]:
moc_df

,Symbol,Imbalance Side,Imbalance Size,Imbalance Reference Price,moc_date
0,AAV,BUY,600,1.585,20200403
1,ABX,BUY,70924,28.275,20200403
2,ACB,BUY,250260,1.125,20200403
3,ACO.X,BUY,16141,35.120,20200403
4,AD,BUY,623,7.450,20200403
...,...,...,...,...,...
393,WFS,SELL,100,1.025,20200403
394,WFT,SELL,437,24.650,20200403
395,WPK,SELL,1631,41.960,20200403
396,X,SELL,18067,104.225,20200403


In [6]:
tsxMocData.output_filename

'moc_tsx_20200403.csv'

In [7]:
tsxMocData.write_tsx_moc(moc_df)

(398, 5)

In [12]:
# Read MOC raw data
raw_moc_flpth = cfg.TSX_MOC_DATA["MOC_DATA_FLPTH"] 

In [19]:
fs = s3fs.S3FileSystem(anon=False)
fs.ls('tsx-moc')

['tsx-moc/moc_20200324.csv',
 'tsx-moc/moc_20200325.csv',
 'tsx-moc/moc_20200326.csv',
 'tsx-moc/moc_20200327.csv',
 'tsx-moc/moc_20200330.csv',
 'tsx-moc/moc_tsx_20200331.csv',
 'tsx-moc/moc_tsx_20200401.csv',
 'tsx-moc/moc_tsx_20200402.csv']

In [17]:
raw_moc_flpth

's3://tsx-moc/'

In [8]:
pre_client = Client()
project_nm = "MOC"

In [9]:
try:
    proj_id = pre_client.create_project(project_name=project_nm)
except Exception as err:
    print(err.with_traceback)

<built-in method with_traceback of ClientError object at 0x7fac5e149b40>


In [10]:
scrape_tsxmoc_fl.storage = Docker(dockerfile="/home/ilivni/MOC/Dockerfile")
scrape_tsxmoc_fl.register(project_name=project_nm)

[2020-04-02 18:41:09,091] INFO - prefect.Docker | Building the flow's Docker storage...
Step 1/16 : FROM python:3.7
 ---> 879165535a54
Step 2/16 : COPY . /extractMOCData
 ---> 8a1f63bb1111
Step 3/16 : WORKDIR /extractMOCData
 ---> Running in 3f6e85618910
Removing intermediate container 3f6e85618910
 ---> 4cdb45a8b3d7
Step 4/16 : RUN apt-get update && apt-get install -y
 ---> Running in 9b1133d6d395
Get:1 http://security.debian.org/debian-security buster/updates InRelease [65.4 kB]
Get:2 http://deb.debian.org/debian buster InRelease [122 kB]
Get:3 http://deb.debian.org/debian buster-updates InRelease [49.3 kB]
Get:4 http://security.debian.org/debian-security buster/updates/main amd64 Packages [186 kB]
Get:5 http://deb.debian.org/debian buster/main amd64 Packages [7907 kB]
Get:6 http://deb.debian.org/debian buster-updates/main amd64 Packages [7380 B]
Fetched 8336 kB in 4s (2146 kB/s)
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information..

ValueError: Your docker image failed to build!  Your flow might have failed one of its deployment health checks - please ensure that all necessary files and dependencies have been included.

In [28]:
Mschedule = IntervalSchedule(
    start_date=datetime.utcnow() + timedelta(seconds=1),
    interval=timedelta(minutes=1),
)
schedule

In [24]:
def retrieve_url(url='https://api.tmxmoney.com/mocimbalance/en/TSX/moc.html'):
    
    html = requests.get(url).content
    df_list = pd.read_html(html, header=[0], displayed_only=False)
    moc_df = df_list[-1]
    
    moc_df["moc_date"] = dt.datetime.today().strftime('%Y%m%d')
    return moc_df

In [25]:
moc_df = moc_to_df()

In [26]:


today = pd.Timestamp('today')
moc_df.to_csv(f"moc_{today:%Y%m%d}.csv", index=False)

In [11]:
moc_df.columns


Index(['Symbol', 'Imbalance Side', 'Imbalance Size',
       'Imbalance Reference Price', 'moc_date'],
      dtype='object')

In [18]:
moc_df["Symbol"] =moc_df["Symbol"].fillna("NA")

In [21]:
moc_df[moc_df["Symbol"].isnull()]

,Symbol,Imbalance Side,Imbalance Size,Imbalance Reference Price,moc_date


In [ ]:
moc_df.to_json(orient="records")